# Beta Analysis

Analyze stock betas vs SPX for beta-neutral hedging.

**Key Concepts:**
- Rolling beta calculation (60-day window)
- Beta-weighted delta for hedging
- Beta stability across time

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.market_data import MarketData

plt.style.use('seaborn-v0_8-whitegrid')
print('Setup complete')

In [ ]:
# Connect
md = MarketData()

## Single Stock Beta Example

In [ ]:
# Calculate beta for a specific stock and date
ticker = 'AAPL'
date = '2002-04-15'

beta = md.get_beta(ticker, date, lookback_days=60)
print(f'{ticker} beta on {date}: {beta:.3f}')

## Beta-Weighted Delta Calculation

In [ ]:
# Example: Calculate beta-weighted delta for a position
# Suppose we're short 10 ATM put contracts on AAPL

position = md.beta_weighted_delta(
    ticker='AAPL',
    delta=-0.5,           # Short ATM put has -0.5 delta
    shares=1000,          # 10 contracts * 100 shares
    stock_price=24.46,    # AAPL price on 2002-04-01
    beta=1.578            # Calculated beta
)

for k, v in position.items():
    print(f'{k}: {v}')

In [ ]:
# Interpretation:
# - Position delta: -$12,230 (short 500 delta-equivalent shares)
# - Beta-weighted: -$19,299 (SPX-equivalent exposure)
# - To hedge: buy ~17 SPX shares (or equivalent ES futures)

## Compare Betas Across Stocks

In [ ]:
# Calculate betas for multiple stocks
tickers = ['AAPL', 'MSFT', 'GE', 'IBM', 'XOM', 'JPM', 'WMT', 'PG']
date = '2002-04-15'

betas = {}
for ticker in tickers:
    try:
        b = md.get_beta(ticker, date, lookback_days=60)
        betas[ticker] = b
        print(f'{ticker}: {b:.3f}')
    except Exception as e:
        print(f'{ticker}: Error - {e}')

In [ ]:
# Plot betas
if betas:
    plt.figure(figsize=(10, 5))
    plt.bar(betas.keys(), betas.values())
    plt.axhline(y=1.0, color='r', linestyle='--', label='Market (beta=1)')
    plt.title(f'Stock Betas vs SPX - {date}')
    plt.xlabel('Ticker')
    plt.ylabel('Beta')
    plt.legend()
    plt.tight_layout()
    plt.show()

## Portfolio Beta-Weighted Delta

For a portfolio of options positions, sum the beta-weighted deltas to get total market exposure.

In [ ]:
# Example portfolio of short strangles
positions = [
    {'ticker': 'AAPL', 'delta': -0.15, 'shares': 1000, 'price': 24.46, 'beta': 1.578},  # Short put
    {'ticker': 'AAPL', 'delta': 0.15, 'shares': 1000, 'price': 24.46, 'beta': 1.578},   # Short call
    {'ticker': 'MSFT', 'delta': -0.15, 'shares': 1000, 'price': 55.0, 'beta': 1.2},     # Short put
    {'ticker': 'MSFT', 'delta': 0.15, 'shares': 1000, 'price': 55.0, 'beta': 1.2},      # Short call
]

total_bwd = 0
print('Position Details:')
print('-' * 60)
for pos in positions:
    bwd = pos['delta'] * pos['shares'] * pos['price'] * pos['beta']
    total_bwd += bwd
    print(f"{pos['ticker']} {'call' if pos['delta'] > 0 else 'put'}: BWD = ${bwd:,.0f}")

print('-' * 60)
print(f'Total Beta-Weighted Delta: ${total_bwd:,.0f}')
print(f'\nTo neutralize: {"Buy" if total_bwd < 0 else "Sell"} ${abs(total_bwd):,.0f} of SPX exposure')

## Notes for Strategy Implementation

1. **Beta estimation**: Use 60-120 day rolling window with shrinkage toward 1.0
2. **Re-hedge triggers**: When |total BWD| exceeds threshold (e.g., 0.2% of portfolio)
3. **Hedge instrument**: ES futures or SPY ETF
4. **Update frequency**: Daily + intraday if threshold breached